### A single script utilizing the Preprocessor, LOOCV_error hyperparameter training, and Prediction methods of the GPyS
#### Note that each of the LOOCV_error and prediction methods has their explicit script which have been directly called in this notebook

#### Last modified: July 18th, 2023.

In [1]:
from GPyS_preprocessor import Preprocessor
from GPyS_LOOCV_error import LOOCV
from GPyS_prediction import Prediction


from scipy.optimize import minimize
import numpy as np

#### Initializing data for example G_12 (the associated physics have been pre-computed - see example_G12.py)


In [2]:
# concatenated orthonormal basis
X = [[0.809017, 0.1045284, -0.6691306,
          -1.0, -0.6691306, 0.1045284,
          0.809017], [0.5877852, 0.9945219,
                      0.7431448, -0.0, -0.7431448,
                      -0.9945219, -0.5877852]]
# sample points
sample = [[0.628319], [1.466077], [2.303835], [3.141593],
              [3.979351], [4.817109], [5.654867]]
# target points
target = [[6.157522]]
# default length scale using rule of thumb --> implemented in LOOCV.default_length() ---> (see section 5 of Zhang et al. (2022) GPS paper)
# the default length scale will be changed to optimal length_scale upon hyperparamter training which will then be used for prediction
length_scale = LOOCV.default_length(1,len(sample))
print(length_scale)

0.42857142857142855


#### compute preprocessing quantities

In [3]:
Preprocessor.setup(X)
XtX = Preprocessor.get_XX_cross_product()
VtX = Preprocessor.get_Vt_X()

# kindly refrain from printing for large matrices
print("XtX:\n", XtX)
print("VtX:\n", VtX)

XtX:
 [[ 0.99999995  0.66913051 -0.10452852 -0.809017   -0.97814755 -0.5
   0.30901706]
 [ 0.66913051  1.          0.66913063 -0.1045284  -0.80901693 -0.97814762
  -0.5       ]
 [-0.10452852  0.66913063  0.99999995  0.6691306  -0.10452843 -0.80901693
  -0.97814755]
 [-0.809017   -0.1045284   0.6691306   1.          0.6691306  -0.1045284
  -0.809017  ]
 [-0.97814755 -0.80901693 -0.10452843  0.6691306   0.99999995  0.66913063
  -0.10452852]
 [-0.5        -0.97814762 -0.80901693 -0.1045284   0.66913063  1.
   0.66913051]
 [ 0.30901706 -0.5        -0.97814755 -0.809017   -0.10452852  0.66913051
   0.99999995]]
VtX:
 [[ 5.87785200e-01  9.94521900e-01  7.43144800e-01  5.39177249e-17
  -7.43144800e-01 -9.94521900e-01 -5.87785200e-01]
 [-8.09017000e-01 -1.04528400e-01  6.69130600e-01  1.00000000e+00
   6.69130600e-01 -1.04528400e-01 -8.09017000e-01]]


#### hyperparameter training (get optimal length scale from LOOCV method)

In [4]:
scale = 2 * np.pi # parameter range for example_G12.py
length = length_scale * scale
LOO = LOOCV(X, sample=sample, beta=[length])
# compute bounds for objective function optimization (discretion of user)
lenUpper = length * 2
lenLower = length * 0.5
# perform optimization by minimizing objective function
ret = minimize(LOO.hSSDist, lenLower, method='Nelder-Mead', bounds=[(lenLower, lenUpper)], tol=1e-6)
print(ret.x)

[objective: 5.48924085468531]; parameter: [1.34639685]
[objective: 5.477454870058285]; parameter: [1.41371669]
[objective: 5.478060783323046]; parameter: [1.48103654]
[objective: 5.47652356373746]; parameter: [1.44737662]
[objective: 5.478060783323046]; parameter: [1.48103654]
[objective: 5.476647213356146]; parameter: [1.43054665]
[objective: 5.477015168634742]; parameter: [1.46420658]
[objective: 5.4765043740606725]; parameter: [1.43896164]
[objective: 5.476647213355775]; parameter: [1.43054665]
[objective: 5.476494247416619]; parameter: [1.44316913]
[objective: 5.47652356373746]; parameter: [1.44737662]
[objective: 5.476494315066677]; parameter: [1.44106538]
[objective: 5.476504040032918]; parameter: [1.44527287]
[objective: 5.476493040560795]; parameter: [1.44211725]
[objective: 5.476494315067294]; parameter: [1.44106538]
[objective: 5.476493334841673]; parameter: [1.44264319]
[objective: 5.476493366618831]; parameter: [1.44159132]
[objective: 5.47649311028643]; parameter: [1.44238

#### compute predictions by calling the principal function from the GPyS_prediction_script.py

In [5]:
length_scale = [1.44213061]

# kindly refrain from printing large matrices
V_circ, sigma2, eps2  = Prediction.GPS_Prediction(
                                    X,
                                    sample,
                                    target,
                                    length_scale,
                                    )
print("Vcirc:\n", V_circ, "\nsigma2:\n", sigma2, "\neps2:\n", eps2)

Vcirc:
 [[-0.39798121 -0.91739357]
 [-0.91739357  0.39798121]] 
sigma2:
 [0.03936761 0.56906575] 
eps2:
 0.007955504984723527
